In [21]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display, HTML
from collections import defaultdict
import itertools
from scipy import stats

# Fix the dying kernel problem (only a problem in some installations - you can remove it, if it works without it)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the dataset for recommenders

In [22]:
data_path = os.path.join("data", "hotel_data")

interactions_df = pd.read_csv(os.path.join(data_path, "hotel_data_interactions_df.csv"), index_col=0)

base_item_features = ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']

column_values_dict = {
    'term': ['WinterVacation', 'Easter', 'OffSeason', 'HighSeason', 'LowSeason', 'MayLongWeekend', 'NewYear', 'Christmas'],
    'length_of_stay_bucket': ['[0-1]', '[2-3]', '[4-7]', '[8-inf]'],
    'rate_plan': ['Standard', 'Nonref'],
    'room_segment': ['[0-160]', '[160-260]', '[260-360]', '[360-500]', '[500-900]'],
    'n_people_bucket': ['[1-1]', '[2-2]', '[3-4]', '[5-inf]'],
    'weekend_stay': ['True', 'False']
}

interactions_df['term'] = pd.Categorical(
    interactions_df['term'], categories=column_values_dict['term'])
interactions_df['length_of_stay_bucket'] = pd.Categorical(
    interactions_df['length_of_stay_bucket'], categories=column_values_dict['length_of_stay_bucket'])
interactions_df['rate_plan'] = pd.Categorical(
    interactions_df['rate_plan'], categories=column_values_dict['rate_plan'])
interactions_df['room_segment'] = pd.Categorical(
    interactions_df['room_segment'], categories=column_values_dict['room_segment'])
interactions_df['n_people_bucket'] = pd.Categorical(
    interactions_df['n_people_bucket'], categories=column_values_dict['n_people_bucket'])
interactions_df['weekend_stay'] = interactions_df['weekend_stay'].astype('str')
interactions_df['weekend_stay'] = pd.Categorical(
    interactions_df['weekend_stay'], categories=column_values_dict['weekend_stay'])

display(interactions_df.head(15))

,user_id,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,2,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,3,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,4,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,5,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
5,6,5,Easter,[4-7],Standard,[260-360],[5-inf],True
6,7,6,OffSeason,[2-3],Standard,[260-360],[5-inf],True
7,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True
8,9,8,HighSeason,[2-3],Standard,[0-160],[1-1],True
9,8,7,HighSeason,[2-3],Standard,[160-260],[1-1],True


# Define user features based on reservations

The content-based recommenders will be forecasting the probability of interaction between user and item based on user features vector and item features vector:

<center>
$$
    r_{u, i} = f(user\_features, item\_features)
$$
</center>

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Design numerical user features based on user reservations. Code the following method which for a given interactions DataFrame (it will be used in the fit method of the recommender) returns a DataFrame with user_id and user features as well as a list with names of user features (this will be important to select the right columns for an ML algorithm). Remember to name the columns differently than item features which you will create in the next task. Validate your features on users with several interactions (sample user ids are already given below).

Ideas for user features:
- Find the vector of most popular feature values from all user reservations and encode every feature with one-hot encoding.
- For every reservation feature calculate the probability distribution of its values among all user's reservations.
- For numerical buckets (length_of_stay, room_segment, n_people) you can calculate the average value for every user from their reservations (you will have to map the buckets back to numerical values before averaging them).

Remember that you will have to select the best features (with the highest explanatory power). Using all above features at once would make the number of variables too large for this dataset and would also introduce too much correlations between features.

You can also prepare several versions of the prepare_users_df method and test which works best in your recommender.

In [356]:
def probability_distribution(interactions_df):
    # prepare result DataFrame template
    users_df = pd.DataFrame(interactions_df.loc[:, "user_id"].unique(), columns=['user_id'])

    inter_df_copy = interactions_df.copy()
    inter_df_copy['val'] = 1
    
    # Calculate the probability distribution for every possible choice
    for bucket_feature in ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']:
        feature_df = inter_df_copy.loc[:, ['user_id', bucket_feature, 'val']]
        feature_df = inter_df_copy.pivot_table(index='user_id', columns=bucket_feature, values='val', aggfunc='count')
        feature_df = feature_df / feature_df.sum(axis=1).values.reshape(-1, 1)  # Summed choices dive by row sum which is equal to number of interacion of each user
        feature_df = feature_df.rename_axis(None, axis=1).fillna(0)
        
        # Make sure that all choices are included, if not add them as new column with deffault value set to 0
        missing_elements = list(set(inter_df_copy[bucket_feature].unique().categories.tolist())-set(inter_df_copy[bucket_feature].unique()))
        feature_df[missing_elements] = 0
        
        feature_df = feature_df.add_prefix('user_' + bucket_feature + '_')
        users_df = pd.merge(users_df, feature_df, on='user_id', how="left")

    # Calculate the average of numerical buckets choice for every user
    numerical_buckets = {
        'length_of_stay_bucket': {'[0-1]': 0.5, '[2-3]': 1.5, '[4-7]': 5.5, '[8-inf]': 10},
        'room_segment': {'[0-160]': 80, '[160-260]': 210, '[260-360]': 310, '[360-500]': 450, '[500-900]': 700},
        'n_people_bucket': {'[1-1]': 1, '[2-2]': 2, '[3-4]': 3.5, '[5-inf]': 7},
    }

    for numerical_feature in numerical_buckets:
        tmp_df = pd.DataFrame(interactions_df.loc[:, "user_id"].unique(), columns=['user_id'])
        
        inter_df_copy.loc[:, numerical_feature] = inter_df_copy.loc[:, numerical_feature].apply(lambda choice: numerical_buckets[numerical_feature][choice]).astype(np.float64)
        
        # reset_index() because otherwise it's assign values to new column by associating tmp_df.index with column index
        new_column_name = "mean_" + numerical_feature
        tmp_df.loc[:, new_column_name] = inter_df_copy.groupby(by="user_id")[numerical_feature].apply(lambda value: np.mean(value)).reset_index(drop=True)

        users_df = pd.merge(users_df, tmp_df, on="user_id", how="left")

        # Fix bug - map_npeople_to_npeople_buckets return None wich is next skipped by pd.pivot_table which cause loss item_id and add NaN during merging
        users_df = users_df.rename_axis(None, axis=1).fillna(0)

    users_df = users_df.reset_index(drop=True)

    return users_df
    
def prepare_users_df(interactions_df):
    # Case II with avg of numerical values
    users_df = probability_distribution(interactions_df)
    
    user_features = users_df.columns[1:].tolist()
    
    return users_df, user_features
    

users_df, user_features = prepare_users_df(interactions_df)

print(user_features)

display(users_df.loc[users_df['user_id'].isin([706, 1736, 7779, 96, 1, 50, 115])].head(15))

['user_term_WinterVacation', 'user_term_Easter', 'user_term_OffSeason', 'user_term_HighSeason', 'user_term_LowSeason', 'user_term_MayLongWeekend', 'user_term_NewYear', 'user_term_Christmas', 'user_length_of_stay_bucket_[0-1]', 'user_length_of_stay_bucket_[2-3]', 'user_length_of_stay_bucket_[4-7]', 'user_length_of_stay_bucket_[8-inf]', 'user_rate_plan_Standard', 'user_rate_plan_Nonref', 'user_room_segment_[0-160]', 'user_room_segment_[160-260]', 'user_room_segment_[260-360]', 'user_room_segment_[360-500]', 'user_room_segment_[500-900]', 'user_n_people_bucket_[1-1]', 'user_n_people_bucket_[2-2]', 'user_n_people_bucket_[3-4]', 'user_n_people_bucket_[5-inf]', 'user_weekend_stay_True', 'user_weekend_stay_False', 'mean_length_of_stay_bucket', 'mean_room_segment', 'mean_n_people_bucket']


,user_id,user_term_WinterVacation,user_term_Easter,user_term_OffSeason,user_term_HighSeason,user_term_LowSeason,user_term_MayLongWeekend,user_term_NewYear,user_term_Christmas,user_length_of_stay_bucket_[0-1],...,user_room_segment_[500-900],user_n_people_bucket_[1-1],user_n_people_bucket_[2-2],user_n_people_bucket_[3-4],user_n_people_bucket_[5-inf],user_weekend_stay_True,user_weekend_stay_False,mean_length_of_stay_bucket,mean_room_segment,mean_n_people_bucket
0,1,0.090909,0.0,0.681818,0.090909,0.136364,0.000000,0.000000,0.0,0.000000,...,0,0.000000,0.727273,0.181818,0.090909,0.772727,0.227273,3.136364,223.636364,2.727273
40,50,0.043478,0.0,0.434783,0.304348,0.217391,0.000000,0.000000,0.0,0.000000,...,0,0.000000,0.173913,0.521739,0.304348,0.782609,0.217391,1.847826,253.478261,4.304348
321,706,0.095238,0.0,0.511905,0.190476,0.142857,0.047619,0.011905,0.0,0.297619,...,0,0.120482,0.156627,0.590361,0.132530,0.583333,0.416667,1.500000,210.000000,3.500000
547,115,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0,0.500000,0.000000,0.000000,0.500000,0.000000,1.000000,1.500000,210.000000,2.000000
1270,1736,0.034483,0.0,0.482759,0.206897,0.275862,0.000000,0.000000,0.0,0.241379,...,0,0.379310,0.413793,0.206897,0.000000,0.448276,0.551724,5.500000,210.000000,3.500000
2785,96,0.090909,0.0,0.681818,0.136364,0.045455,0.045455,0.000000,0.0,0.272727,...,0,0.045455,0.272727,0.590909,0.090909,0.727273,0.272727,0.500000,210.000000,3.500000
11527,7779,0.000000,0.0,0.500000,0.000000,0.500000,0.000000,0.000000,0.0,0.000000,...,0,0.000000,0.000000,0.750000,0.250000,0.750000,0.250000,1.500000,80.000000,2.000000


# Prepare numerical item features

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the prepare_items_df method which will be used in the recommender fit and recommend methods to map items to numerical features. This method should take the interactions_df DataFrame as input and return a DataFrame containing one record per item_id with item_id column and numerical item feature columns.

You can try turning all item features into one-hot representations. You can use the get_dummies method from pandas. It will return the same columns on any dataset of interactions because the categorical variables with all possible values have been defined in the second cell in this notebook.

You are welcome to design your own numerical item features, for instance based on numerical min and max values in buckets used as features.

In [357]:
def item_numerical_features(interactions_df):
    # prepare result DataFrame template
    item_df = pd.DataFrame(interactions_df.loc[:, "item_id"].unique(), columns=['item_id'])

    inter_df_copy = interactions_df.copy()
    inter_df_copy['val'] = 1
    
    # Calculate the probability distribution for every possible choice
    for bucket_feature in ['term', 'length_of_stay_bucket', 'rate_plan', 'room_segment', 'n_people_bucket', 'weekend_stay']:
        feature_df = inter_df_copy.loc[:, ['item_id', bucket_feature, 'val']]
        feature_df = inter_df_copy.pivot_table(index='item_id', columns=bucket_feature, values='val')
        feature_df = feature_df / feature_df.sum(axis=1).values.reshape(-1, 1)  # Summed choices dive by row sum which is equal to number of interacion of each user
        feature_df = feature_df.rename_axis(None, axis=1).fillna(0)

        # Make sure that all choices are included, if not add them as new column with deffault value set to 0
        missing_elements = list(set(inter_df_copy[bucket_feature].unique().categories.tolist())-set(inter_df_copy[bucket_feature].unique()))
        feature_df[missing_elements] = 0
        
        feature_df = feature_df.add_prefix('item_' + bucket_feature + '_')
        item_df = pd.merge(item_df, feature_df, on='item_id', how="left")

        # Fix bug - map_npeople_to_npeople_buckets return None wich is next skipped by pd.pivot_table which cause loss item_id and add NaN during merging
        item_df = item_df.rename_axis(None, axis=1).fillna(0)

    # Calculate the popularity of each item
#     item_popularity = inter_df_copy.groupby(by='item_id', as_index=False)['val'].count().reset_index(drop=True).copy()
#     item_popularity['val'] = item_popularity['val'].apply(lambda value: np.divide(value, 100))
#     item_popularity = item_popularity.rename(columns={'val': 'item_popularity'})
    
#     item_df = pd.merge(item_df, item_popularity, on='item_id', how='left')

    item_df = item_df.reset_index(drop=True)

    return item_df

def prepare_items_df(interactions_df):
    
    items_df = item_numerical_features(interactions_df)
    item_features = items_df.columns[1:].tolist()

    return items_df, item_features

items_df, item_features = prepare_items_df(interactions_df)

print(item_features)

display(items_df.loc[items_df['item_id'].isin([0, 1, 2, 3, 4, 5, 6])].head(15))

['item_term_WinterVacation', 'item_term_Easter', 'item_term_OffSeason', 'item_term_HighSeason', 'item_term_LowSeason', 'item_term_MayLongWeekend', 'item_term_NewYear', 'item_term_Christmas', 'item_length_of_stay_bucket_[0-1]', 'item_length_of_stay_bucket_[2-3]', 'item_length_of_stay_bucket_[4-7]', 'item_length_of_stay_bucket_[8-inf]', 'item_rate_plan_Standard', 'item_rate_plan_Nonref', 'item_room_segment_[0-160]', 'item_room_segment_[160-260]', 'item_room_segment_[260-360]', 'item_room_segment_[360-500]', 'item_room_segment_[500-900]', 'item_n_people_bucket_[1-1]', 'item_n_people_bucket_[2-2]', 'item_n_people_bucket_[3-4]', 'item_n_people_bucket_[5-inf]', 'item_weekend_stay_True', 'item_weekend_stay_False']


,item_id,item_term_WinterVacation,item_term_Easter,item_term_OffSeason,item_term_HighSeason,item_term_LowSeason,item_term_MayLongWeekend,item_term_NewYear,item_term_Christmas,item_length_of_stay_bucket_[0-1],...,item_room_segment_[160-260],item_room_segment_[260-360],item_room_segment_[360-500],item_room_segment_[500-900],item_n_people_bucket_[1-1],item_n_people_bucket_[2-2],item_n_people_bucket_[3-4],item_n_people_bucket_[5-inf],item_weekend_stay_True,item_weekend_stay_False
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0
1,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0,0.0,0.0,1.0,0.0,1.0,0.0
2,2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0,0.0,1.0,0.0,0.0,0.0,1.0
3,3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0,0.0,0.0,1.0,0.0,1.0,0.0
4,4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0.0,1.0,0.0,0.0,1.0,0.0
5,5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0
6,6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0,0.0,0.0,0.0,1.0,1.0,0.0


# Content-based recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Code the content-based recommender. User features should be calculated within the fit method based on available training data and should be saved in the object as self.users_df for later use in the recommend method. Item features should be calculated both in the fit method (from interactions_df) and in the recommend method (from items_df - the items to be evaluated).

In the fit method you have to randomly generate non-existing interactions and add them to the training data for the regressor. You should add the target variable to interactions - equal to 1 for real ("positive") interactions and equal to 0 for those newly added "negative" interactions. Generate several negative interactions per every positive interaction (n_neg_per_pos). Treat the proportion as a tunable parameter of the model.

Remember to keep control over randomness - in the init method add seed as a parameter and initialize the random seed generator with that seed:

```python
self.seed = seed
self.rng = np.random.RandomState(seed=seed)
```

Below the base content-based recommender class there are several classes which inherit from the base class and use different ML models:
  - LinearRegressionCBUIRecommender - based on linear regression,
  - SVRCBUIRecommender - based on Support Vector Regressor (if you want to test it, sample the data in the fit method, as the training can take many hours on the entire dataset of interactions),
  - RandomForestCBUIRecommender - based on Random Forest,
  - XGBoostCBUIRecommender - based on XGBoost.
  
There is no need to change anything in those inheriting classes, although you can experiment with other tunable parameters of the underlying models.

You are encouraged to experiment with:
  - Other numerical user and item features (but always train and evaluate the model on buckets defined in the first notebook).
  - Other ML models, e.g. Huber regression, Lasso regression, Ridge regression, LARS regression, Linear SVR, Decision Tree, Naive Bayes, LightGBM, Neural Networks or any model of your choice.

In [358]:
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from recommenders.recommender import Recommender


class ContentBasedUserItemRecommender(Recommender):
    """
    Linear recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5):
        """
        Initialize base recommender params and variables.
        """
        self.model = LinearRegression()
        self.n_neg_per_pos = n_neg_per_pos
        
        self.recommender_df = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        self.users_df = None
        self.user_features = None
        
        self.seed = seed
        self.rng = np.random.RandomState(seed=seed)
    
    def fit(self, interactions_df, users_df, items_df):
        """
        Training of the recommender.
        
        :param pd.DataFrame interactions_df: DataFrame with recorded interactions between users and items 
            defined by user_id, item_id and features of the interaction.
        :param pd.DataFrame users_df: DataFrame with users and their features defined by user_id and the user feature columns.
        :param pd.DataFrame items_df: DataFrame with items and their features defined by item_id and the item feature columns.
        """
        
        interactions_df = interactions_df.copy()
        
        # Prepare users_df and items_df
        
        users_df, user_features = prepare_users_df(interactions_df)
        
        self.users_df = users_df
        self.user_features = user_features
        
        items_df, item_features = prepare_items_df(interactions_df)
        items_df = items_df.loc[:, ['item_id'] + item_features]
        
        # Generate negative interactions
        
        interactions_df = interactions_df.loc[:, ['user_id', 'item_id']]
        
        interactions_df.loc[:, 'interacted'] = 1
        
        negative_interactions = []
        
        ########################
        # Write your code here #
        # Generate tuples (user_id, item_id, 0) for pairs (user_id, item_id) which do not
        # appear in the interactions_df and add those tuples to the list negative_interactions.
        # Generate self.n_neg_per_pos * len(interactions_df) negative interactions 
        # (self.n_neg_per_pos per one positive).
        # Make sure the code is efficient and runs fast, otherwise you will not be able to properly tune your model.
        user_id = interactions_df['user_id'].unique().tolist()
        item_id = interactions_df['item_id'].unique().tolist()

        touple_of_all_combinations = list(itertools.product(user_id, item_id))  # list of all combinations
        touple_of_interactions_pair = [tuple(x) for x in interactions_df.loc[:, ['user_id', 'item_id']].to_numpy()]  # list of interactions combinations 
        not_appear_combinations = list(set(touple_of_all_combinations) - set(touple_of_interactions_pair))  # defference of sets get combinations not appear in interactions_df

        rng = np.random.RandomState(seed=self.seed)
        random_ids_combinations = rng.choice(list(range(len(not_appear_combinations))), size=int(self.n_neg_per_pos) * len(interactions_df), replace=False) # generate set of negative interactions
        negative_interactions = [not_appear_combinations[index] for index in random_ids_combinations]  # map index to its value
        negative_interactions = [list(tuple_combination)+[0] for tuple_combination in negative_interactions]  # change strusture of touple, add 0 as sufix

        interactions_df = pd.concat(
            [interactions_df, pd.DataFrame(negative_interactions, columns=['user_id', 'item_id', 'interacted'])])
        
        # Get the input data for the model
        
        interactions_df = pd.merge(interactions_df, users_df, on=['user_id'])
        interactions_df = pd.merge(interactions_df, items_df, on=['item_id'])
        
#         display('user_features: ' + str(len(user_features)))
#         display(user_features)
#         display('item_features: ' + str(len(item_features)))
#         display(item_features)
        
        x = interactions_df.loc[:, user_features + item_features].values
        y = interactions_df['interacted'].values

        self.model.fit(x, y)
    
    def recommend(self, users_df, items_df, n_recommendations=1):
        """
        Serving of recommendations. Scores items in items_df for each user in users_df and returns 
        top n_recommendations for each user.
        
        :param pd.DataFrame users_df: DataFrame with users and their features for which recommendations should be generated.
        :param pd.DataFrame items_df: DataFrame with items and their features which should be scored.
        :param int n_recommendations: Number of recommendations to be returned for each user.
        :return: DataFrame with user_id, item_id and score as columns returning n_recommendations top recommendations 
            for each user.
        :rtype: pd.DataFrame
        """
        
        # Clean previous recommendations (iloc could be used alternatively)
        self.recommender_df = self.recommender_df[:0]
        
        ########################
        # Write your code here #
        # Prepare users_df and items_df
        # For users_df you need to merge user features from self.users_df to users_df 
        # (the users for which you generate recommendations).
        # Note that for users who were not in the hotel before (which is true for most users)
        # there will be no features in self.users_df. For such users you can initialize their features
        # with all zeros (for instance with fillna(0)), but you can also try to use average feature
        # values from self.users_df (this way you would trear a new user as an average user).
        # For items you have to apply the prepare_items_df method to items_df.
        
        # prepare average features
        recommend_df = items_df.copy()
        
        users_df = pd.merge(users_df, self.users_df, on='user_id', how='left')
        users_df = users_df.fillna(self.users_df.mean())
        
        items_df, item_features = prepare_items_df(items_df)
        
        # Score the items
        
        recommendations = pd.DataFrame(columns=['user_id', 'item_id', 'score'])
        
        for ix, user in users_df.iterrows():
            ########################
            # Write your code here #
            # Create a Carthesian product of users from users_df and items from items_df
            carthesian_product = pd.merge(items_df, user.to_frame().transpose(), how='cross')
            carthesian_product = carthesian_product.drop(['user_id', 'item_id'], axis=1)

            ########################
            # Write your code here #
            # Use self.model.predict method to calculate scores for all records in the just created DataFrame
            # of users and items
            scores = self.model.predict(carthesian_product.loc[:, self.user_features + item_features].values)

            ########################
            # Write your code here #
            # Obtain item ids with the highest score and save those ids under the chosen_ids variable
            # Do not exclude already booked items.
            chosen_ids = np.argsort(-scores)[:n_recommendations]
            
            recommendations = []
            for item_id in chosen_ids:
                recommendations.append(
                    {
                        'user_id': user['user_id'],
                        'item_id': item_id,
                        'score': scores[item_id]
                    }
                )
            
            user_recommendations = pd.DataFrame(recommendations)

            self.recommender_df = pd.concat([self.recommender_df, user_recommendations])

        return self.recommender_df
    
    
class LinearRegressionCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Linear regression recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        self.model = LinearRegression()
        
        
class SVRCBUIRecommender(ContentBasedUserItemRecommender):
    """
    SVR recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'kernel' in model_params:
            self.kernel = model_params['kernel']
        else:
            self.kernel = 'rbf'
        if 'C' in model_params:
            self.C = model_params['C']
        else:
            self.C = 1.0
        if 'epsilon' in model_params:
            self.epsilon = model_params['epsilon']
        else:
            self.epsilon = 0.1
        self.model = SVR(kernel=self.kernel, C=self.C, epsilon=self.epsilon)
        
    
class RandomForestCBUIRecommender(ContentBasedUserItemRecommender):
    """
    Random forest recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        self.model = RandomForestRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split)
    
    
class XGBoostCBUIRecommender(ContentBasedUserItemRecommender):
    """
    XGBoost recommender class based on user and item features.
    """
    
    def __init__(self, seed=6789, n_neg_per_pos=5, **model_params):
        """
        Initialize base recommender params and variables.
        """
        super().__init__(seed=seed, n_neg_per_pos=n_neg_per_pos)
        if 'n_estimators' in model_params:
            self.n_estimators = int(model_params['n_estimators'])
        else:
            self.n_estimators = 100
        if 'max_depth' in model_params:
            self.max_depth = int(model_params['max_depth'])
        else:
            self.max_depth = 30
        if 'min_samples_split' in model_params:
            self.min_samples_split = int(model_params['min_samples_split'])
        else:
            self.min_samples_split = 30
        if 'learning_rate' in model_params:
            self.learning_rate = model_params['learning_rate']
        else:
            self.learning_rate = 30
        self.model = GradientBoostingRegressor(
            n_estimators=self.n_estimators, max_depth=self.max_depth, min_samples_split=self.min_samples_split,
            learning_rate=self.learning_rate)    

# Quick test of the recommender

In [359]:
items_df = interactions_df.loc[:, ['item_id'] + base_item_features].drop_duplicates()
display(items_df)

,item_id,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,0,WinterVacation,[2-3],Standard,[260-360],[5-inf],True
1,1,WinterVacation,[2-3],Standard,[160-260],[3-4],True
2,2,WinterVacation,[2-3],Standard,[160-260],[2-2],False
3,3,WinterVacation,[4-7],Standard,[160-260],[3-4],True
4,4,WinterVacation,[4-7],Standard,[0-160],[2-2],True
...,...,...,...,...,...,...,...
15079,758,OffSeason,[2-3],Nonref,[0-160],[3-4],True
15176,759,Christmas,[8-inf],Standard,[0-160],[2-2],True
15186,760,NewYear,[4-7],Standard,[160-260],[1-1],False
15191,761,NewYear,[4-7],Nonref,[260-360],[3-4],True


In [334]:
# Fit method
cb_user_item_recommender = RandomForestCBUIRecommender()
cb_user_item_recommender.fit(interactions_df, None, None)

'user_features: 28'

['user_term_WinterVacation',
 'user_term_Easter',
 'user_term_OffSeason',
 'user_term_HighSeason',
 'user_term_LowSeason',
 'user_term_MayLongWeekend',
 'user_term_NewYear',
 'user_term_Christmas',
 'user_length_of_stay_bucket_[0-1]',
 'user_length_of_stay_bucket_[2-3]',
 'user_length_of_stay_bucket_[4-7]',
 'user_length_of_stay_bucket_[8-inf]',
 'user_rate_plan_Standard',
 'user_rate_plan_Nonref',
 'user_room_segment_[0-160]',
 'user_room_segment_[160-260]',
 'user_room_segment_[260-360]',
 'user_room_segment_[360-500]',
 'user_room_segment_[500-900]',
 'user_n_people_bucket_[1-1]',
 'user_n_people_bucket_[2-2]',
 'user_n_people_bucket_[3-4]',
 'user_n_people_bucket_[5-inf]',
 'user_weekend_stay_True',
 'user_weekend_stay_False',
 'mean_length_of_stay_bucket',
 'mean_room_segment',
 'mean_n_people_bucket']

'item_features: 26'

['item_term_WinterVacation',
 'item_term_Easter',
 'item_term_OffSeason',
 'item_term_HighSeason',
 'item_term_LowSeason',
 'item_term_MayLongWeekend',
 'item_term_NewYear',
 'item_term_Christmas',
 'item_length_of_stay_bucket_[0-1]',
 'item_length_of_stay_bucket_[2-3]',
 'item_length_of_stay_bucket_[4-7]',
 'item_length_of_stay_bucket_[8-inf]',
 'item_rate_plan_Standard',
 'item_rate_plan_Nonref',
 'item_room_segment_[0-160]',
 'item_room_segment_[160-260]',
 'item_room_segment_[260-360]',
 'item_room_segment_[360-500]',
 'item_room_segment_[500-900]',
 'item_n_people_bucket_[1-1]',
 'item_n_people_bucket_[2-2]',
 'item_n_people_bucket_[3-4]',
 'item_n_people_bucket_[5-inf]',
 'item_weekend_stay_True',
 'item_weekend_stay_False',
 'item_popularity']

In [335]:
# Recommender method

recommendations = cb_user_item_recommender.recommend(pd.DataFrame([[1], [2], [3], [4], [5]], columns=['user_id']), interactions_df, 10)

recommendations = pd.merge(recommendations, items_df, on='item_id', how='left')
display(recommendations)

,user_id,item_id,score,term,length_of_stay_bucket,rate_plan,room_segment,n_people_bucket,weekend_stay
0,1.0,55,0.993421,OffSeason,[2-3],Nonref,[160-260],[2-2],True
1,1.0,61,0.990121,OffSeason,[4-7],Nonref,[160-260],[2-2],True
2,1.0,23,0.988749,OffSeason,[4-7],Standard,[160-260],[2-2],True
3,1.0,51,0.985393,OffSeason,[2-3],Nonref,[160-260],[3-4],True
4,1.0,14,0.979352,OffSeason,[4-7],Standard,[160-260],[3-4],True
5,1.0,22,0.979152,OffSeason,[2-3],Standard,[160-260],[3-4],True
6,1.0,76,0.97202,OffSeason,[4-7],Nonref,[160-260],[3-4],True
7,1.0,32,0.970031,OffSeason,[2-3],Standard,[160-260],[2-2],True
8,1.0,142,0.931873,LowSeason,[2-3],Nonref,[160-260],[2-2],True
9,1.0,21,0.91539,OffSeason,[2-3],Standard,[160-260],[2-2],False


# Tuning method

In [360]:
from evaluation_and_testing.testing import evaluate_train_test_split_implicit

seed = 6789

In [361]:
from hyperopt import hp, fmin, tpe, Trials
import traceback

def tune_recommender(recommender_class, interactions_df, items_df, 
                     param_space, max_evals=1, show_progressbar=True, seed=6789):
    # Split into train_validation and test sets

    shuffle = np.arange(len(interactions_df))
    rng = np.random.RandomState(seed=seed)
    rng.shuffle(shuffle)
    shuffle = list(shuffle)

    train_test_split = 0.8
    split_index = int(len(interactions_df) * train_test_split)

    train_validation = interactions_df.iloc[shuffle[:split_index]]
    test = interactions_df.iloc[shuffle[split_index:]]

    # Tune

    def loss(tuned_params):
        recommender = recommender_class(seed=seed, **tuned_params)
        hr1, hr3, hr5, hr10, ndcg1, ndcg3, ndcg5, ndcg10 = evaluate_train_test_split_implicit(
            recommender, train_validation, items_df, seed=seed)
        return -hr10

    n_tries = 1
    succeded = False
    try_id = 0
    while not succeded and try_id < n_tries:
        try:
            trials = Trials()
            best_param_set = fmin(loss, space=param_space, algo=tpe.suggest, 
                                  max_evals=max_evals, show_progressbar=show_progressbar, trials=trials, verbose=True)
            succeded = True
        except:
            traceback.print_exc()
            try_id += 1
            
    if not succeded:
        return None
        
    # Validate
    
    recommender = recommender_class(seed=seed, **best_param_set)

    results = [[recommender_class.__name__] + list(evaluate_train_test_split_implicit(
        recommender, {'train': train_validation, 'test': test}, items_df, seed=seed))]

    results = pd.DataFrame(results, 
                           columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

    display(results)
    
    return best_param_set

## Tuning of the recommender

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Tune your models using the code below. You only need to put the class name of your recommender and choose an appropriate parameter space.

In [362]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1)
}

best_param_set = tune_recommender(LinearRegressionCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=10, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|                                                                           | 0/10 [00:33<?, ?trial/s, best loss=?]
Best parameters:
None


Traceback (most recent call last):
  File "C:\Users\Windows 10\AppData\Local\Temp\ipykernel_9220\3138901096.py", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 540, in fmin
    return trials.fmin(
  File "d:\lang\python39\lib\site-packages\hyperopt\base.py", line 671, in fmin
    return fmin(
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 586, in fmin
    rval.exhaust()
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 300, in run
    self.serial_evaluate()
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 178, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "d:\lang\python39\lib\site-packages\hyperopt\base.py", line 892, in evaluate
    r

In [29]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'C': hp.loguniform('C', np.log(0.01), np.log(100.0))
}

best_param_set = tune_recommender(SVRCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

  0%|                                                                          | 0/100 [00:03<?, ?trial/s, best loss=?]
Best parameters:
None


Traceback (most recent call last):
  File "C:\Users\Windows 10\AppData\Local\Temp\ipykernel_9220\3138901096.py", line 33, in tune_recommender
    best_param_set = fmin(loss, space=param_space, algo=tpe.suggest,
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 540, in fmin
    return trials.fmin(
  File "d:\lang\python39\lib\site-packages\hyperopt\base.py", line 671, in fmin
    return fmin(
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 586, in fmin
    rval.exhaust()
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 364, in exhaust
    self.run(self.max_evals - n_done, block_until_done=self.asynchronous)
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 300, in run
    self.serial_evaluate()
  File "d:\lang\python39\lib\site-packages\hyperopt\fmin.py", line 178, in serial_evaluate
    result = self.domain.evaluate(spec, ctrl)
  File "d:\lang\python39\lib\site-packages\hyperopt\base.py", line 892, in evaluate
    r

In [18]:
param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 30, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1)
}

best_param_set = tune_recommender(RandomForestCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=100, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

100%|█████████████████████████████████████████| 100/100 [9:24:26<00:00, 338.67s/trial, best loss: -0.12484130342784595]


,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.001697,0.068907,0.078751,0.125594,0.001697,0.039969,0.043926,0.058691


Best parameters:
{'max_depth': 7.0, 'min_samples_split': 19.0, 'n_estimators': 300.0, 'n_neg_per_pos': 3.0}


In [ ]:
# This tuning may take around 12 hours

param_space = {
    'n_neg_per_pos': hp.quniform('n_neg_per_pos', 1, 10, 1),
    'n_estimators': hp.quniform('n_estimators', 10, 300, 1),
    'max_depth': hp.quniform('max_depth', 2, 10, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 30, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.1))
}

best_param_set = tune_recommender(XGBoostCBUIRecommender, interactions_df, items_df,
                                  param_space, max_evals=300, show_progressbar=True, seed=seed)

print("Best parameters:")
print(best_param_set)

# Final evaluation

<span style="color:red"><font size="4">**Task:**</font></span><br> 
Run the final evaluation of your recommender and present its results against the Amazon recommender's results. You can present results for several of your recommenders. You just need to give the class name of your recommender and its tuned parameters below. If you present results for several recommenders, you should add a separate cell for each recommender and change the names of the DataFrames containing results.

In [363]:
cb_user_item_recommender = LinearRegressionCBUIRecommender(
    **{'n_neg_per_pos': 10})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
linear_cbui_tts_results = [['LinearRegressionCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

linear_cbui_tts_results = pd.DataFrame(
    linear_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(linear_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.228785,0.049219,0.094502,0.113016,0.130493


In [364]:
cb_user_item_recommender = RandomForestCBUIRecommender(
    **{'max_depth': 7.0, 'min_samples_split': 19.0, 'n_estimators': 300.0, 'n_neg_per_pos': 3.0})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
randomForest_cbui_tts_results = [['RandomForestCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

randomForest_cbui_tts_results = pd.DataFrame(
    randomForest_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(randomForest_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,RandomForestCBUIRecommender,0.04854,0.132383,0.175832,0.230482,0.04854,0.095573,0.113779,0.131301


In [366]:
cb_user_item_recommender = XGBoostCBUIRecommender(
    **{'learning_rate': 0.02, 'max_depth': 7.0, 'min_samples_split': 18.0, 'n_estimators': 130.0, 'n_neg_per_pos': 4.0})  # Initialize your recommender here with the best params from tuning

# Give the name of your recommender in the line below
xgboost_cbui_tts_results = [['XGBoostCBUIRecommender'] + list(evaluate_train_test_split_implicit(
    cb_user_item_recommender, interactions_df, items_df))]

xgboost_cbui_tts_results = pd.DataFrame(
    xgboost_cbui_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(xgboost_cbui_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.042091,0.107264,0.162254,0.236253,0.042091,0.081611,0.103434,0.127794


In [365]:
from recommenders.amazon_recommender import AmazonRecommender

amazon_recommender = AmazonRecommender()

amazon_tts_results = [['AmazonRecommender'] + list(evaluate_train_test_split_implicit(
    amazon_recommender, interactions_df, items_df))]

amazon_tts_results = pd.DataFrame(
    amazon_tts_results, columns=['Recommender', 'HR@1', 'HR@3', 'HR@5', 'HR@10', 'NDCG@1', 'NDCG@3', 'NDCG@5', 'NDCG@10'])

display(amazon_tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468


In [368]:
tts_results = pd.concat([xgboost_cbui_tts_results, randomForest_cbui_tts_results, linear_cbui_tts_results, amazon_tts_results]).reset_index(drop=True)
display(tts_results)

,Recommender,HR@1,HR@3,HR@5,HR@10,NDCG@1,NDCG@3,NDCG@5,NDCG@10
0,XGBoostCBUIRecommender,0.042091,0.107264,0.162254,0.236253,0.042091,0.081611,0.103434,0.127794
1,RandomForestCBUIRecommender,0.048540,0.132383,0.175832,0.230482,0.048540,0.095573,0.113779,0.131301
2,LinearRegressionCBUIRecommender,0.049219,0.130007,0.174134,0.228785,0.049219,0.094502,0.113016,0.130493
3,AmazonRecommender,0.044128,0.118805,0.160557,0.223693,0.044128,0.086755,0.104216,0.124468
